In [8]:
ipn_approve = snowflake_query("Egypt", "select * from emoney.bank_deposit_requests", warehouse=None, columns=[], conn=None)
ipn_approve.columns = ipn_approve.columns.str.upper()
ipn_approve.head(5)

/home/ec2-user/service_account_key.json


,ID,CREATED_AT,UPDATED_AT,RETAILER_ID,BANK_ID,CASHIN_ID,LAST_MODIFIED_BY_USER_ID,NONCE,REQUESTED_AMOUNT,RECEIPT_URL,DEPOSIT_REFERENCE_NUMBER,REVIEWED_AMOUNT,REQUEST_STATUS,REJECTION_REASON,REQUEST_TYPE,TOTAL_AMOUNT,FEES,CREATED_BY_USER_ID
0,15186,2025-08-28 12:47:20.583892,2025-08-28 12:47:20.583892,203787,1,16314466,None,37c5e28a-6db7-4542-9f2e-ed0f7a9d064a,1365.00,payment-methods/bank-transfer/retailer-receipt...,None,None,PENDING,None,DEPOSIT,1365.00,3.42,None
1,15182,2025-08-28 12:38:16.509428,2025-08-28 12:38:16.509428,1427401,1,16314315,None,46043a45-bc1d-4776-92a4-e464c0214247,500.00,payment-methods/bank-transfer/retailer-receipt...,None,None,PENDING,None,DEPOSIT,500.00,3.00,None
2,15179,2025-08-28 12:34:29.525781,2025-08-28 12:34:29.525781,1447013,1,16314242,None,e6047034-5fd4-4951-9ff0-767ee0fc40df,500.00,payment-methods/bank-transfer/retailer-receipt...,None,None,PENDING,None,DEPOSIT,500.00,3.00,None
3,15180,2025-08-28 12:35:01.568762,2025-08-28 12:35:01.568762,1278270,1,16314251,None,8cd5a015-2879-4e70-b5e9-c0750cb44e7a,500.00,payment-methods/bank-transfer/retailer-receipt...,None,None,PENDING,None,DEPOSIT,500.00,3.00,None
4,15181,2025-08-28 12:35:19.955963,2025-08-28 12:35:19.955963,1863817,1,16314254,None,0f821910-4060-41cf-b111-29be8586a1df,600.00,payment-methods/bank-transfer/retailer-receipt...,None,None,PENDING,None,DEPOSIT,600.00,3.00,None


In [7]:
ipn_approve = ret_metabase("Egypt", 63680)
ipn_approve.head(5)

/home/ec2-user/service_account_key.json


,ID,CREATED_AT,UPDATED_AT,RETAILER_ID,BANK_ID,CASHIN_ID,LAST_MODIFIED_BY_USER_ID,NONCE,REQUESTED_AMOUNT,RECEIPT_URL,DEPOSIT_REFERENCE_NUMBER,REVIEWED_AMOUNT,REQUEST_STATUS,REJECTION_REASON,REQUEST_TYPE,TOTAL_AMOUNT,FEES,CREATED_BY_USER_ID
0,15174,2025-08-28T12:23:15.803,2025-08-28T12:23:15.803,1234165,1,16314024,NaN,4eeae1bc-1ad4-41a1-a985-80fee34f8e63,1000.0,payment-methods/bank-transfer/retailer-receipt...,NaN,NaN,PENDING,NaN,DEPOSIT,1000.0,3.0,NaN
1,15173,2025-08-28T12:21:45.489,2025-08-28T12:21:45.489,241664,1,16314005,NaN,332a6f43-81e6-4221-b8da-30335aa49880,1000.0,payment-methods/bank-transfer/retailer-receipt...,NaN,NaN,PENDING,NaN,DEPOSIT,1000.0,3.0,NaN
2,15172,2025-08-28T12:20:54.003,2025-08-28T12:20:54.003,1741138,1,16313985,NaN,93fe2c1a-fbf1-4d12-bce3-31f360f99148,500.0,payment-methods/bank-transfer/retailer-receipt...,NaN,NaN,PENDING,NaN,DEPOSIT,500.0,3.0,NaN
3,15171,2025-08-28T12:20:12.916,2025-08-28T12:20:12.916,1817658,1,16313970,NaN,c7de1e81-62c3-4007-a5df-30291c2681a2,503.0,payment-methods/bank-transfer/retailer-receipt...,NaN,NaN,PENDING,NaN,DEPOSIT,503.0,3.0,NaN
4,15170,2025-08-28T12:19:11.59,2025-08-28T12:19:11.59,1723571,1,16313953,NaN,b40753cf-f4ec-4c01-b8d5-5d5aa1f5a53f,500.0,payment-methods/bank-transfer/retailer-receipt...,NaN,NaN,PENDING,NaN,DEPOSIT,500.0,3.0,NaN


In [3]:
from common_functions import ret_metabase, google_sheets, dwh_query, upload_dataframe_to_pg, upload_dataframe_to_snowflake, snowflake_query

import pandas as pd
from datetime import datetime, timedelta
import datetime as dt
import time
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
import requests
from pathlib import Path
from io import StringIO
import pandas as pd
import sqlalchemy
import psycopg2
import numpy as np
import gspread
import pytz

ipn_approve = ret_metabase("Egypt", 63680)
check_data = snowflake_query("Egypt", "select * from materialized_views.IPN_users", warehouse=None, columns=[], conn=None)

# ensure column is in datetime format
bank_df = google_sheets("Test_IPN", "bank", "get")
bank_df = bank_df[['Transaction Date/Time', 'Description','Credit']]    
bank_df['Transaction Date/Time'] = pd.to_datetime(bank_df['Transaction Date/Time'])

# filter last 10 days
last_10_days = pd.Timestamp.today().normalize() - pd.Timedelta(days=10)
filtered_df = bank_df[bank_df['Transaction Date/Time'] >= last_10_days]

# if you only want the date part (not time)
filtered_df['Transaction Date'] = filtered_df['Transaction Date/Time'].dt.date

# keep only the needed columns
filtered_df = filtered_df[['Transaction Date', 'Description', 'Credit']]

approve_list = ipn_approve[ipn_approve["REQUEST_STATUS"] ==  "APPROVED"]
approve_list = approve_list[["ID", 'CREATED_AT','RETAILER_ID', 'REQUEST_STATUS', 'REQUESTED_AMOUNT']]

to_approve_list = ipn_approve[ipn_approve["REQUEST_STATUS"] ==  "PENDING"]
to_approve_list = to_approve_list[["ID",'RETAILER_ID', 'REQUEST_STATUS','NONCE', 'REQUESTED_AMOUNT']]


# parse CREATED_AT safely (handles both with/without milliseconds)
approve_list['CREATED_AT'] = pd.to_datetime(approve_list['CREATED_AT'], format="ISO8601", errors="coerce")

# calculate 10 days ago
last_10_days = pd.Timestamp.today().normalize() - pd.Timedelta(days=10)

# filter last 10 days
approve_filtered = approve_list[approve_list['CREATED_AT'] >= last_10_days]

# extract only date
approve_filtered['CREATED_DATE'] = approve_filtered['CREATED_AT'].dt.date

# keep only the needed columns
approve_filtered = approve_filtered[["ID",'CREATED_DATE','RETAILER_ID', 'REQUEST_STATUS', 'REQUESTED_AMOUNT']]

to_approve_list['RETAILER_ID'] = to_approve_list['RETAILER_ID'].astype(str)
check_data['retailer_id'] = check_data['retailer_id'].astype(str)
# Retailers that ARE in check_data
in_check_data = to_approve_list[to_approve_list['RETAILER_ID'].isin(check_data['retailer_id'])]

# Retailers that are NOT in check_data
not_in_check_data = to_approve_list[~to_approve_list['RETAILER_ID'].isin(check_data['retailer_id'])] ### add this to manual approval
not_in_check_data["Reason"] = 'First Time'
google_sheets("Test_IPN", "Manual", "append", df=not_in_check_data)
# Check the results
print(f"Total retailers in approve_list: {len(to_approve_list)}")
print(f"Retailers in check_data: {len(in_check_data)}")
print(f"Retailers NOT in check_data: {len(not_in_check_data)}")

checking_data = in_check_data.merge(
    check_data[['retailer_id', 'extracted_name']], 
    left_on='RETAILER_ID', 
    right_on='retailer_id', 
    how='left'
)

# Group by RETAILER_ID and combine extracted names
check_data_with_names = checking_data.groupby('RETAILER_ID').agg({
    'extracted_name': lambda x: ' | '.join(x.dropna().unique())
}).reset_index()

# Iterate through each row for processing
for _, row in check_data_with_names.iterrows():
    retailer_id = row['RETAILER_ID']
    extracted_name = row['extracted_name']
    
    print(f"Processing retailer ID: {retailer_id}")
    print(f"Extracted names: {extracted_name}")
    
    # Create a single-row DataFrame for this retailer
    single_retailer_df = pd.DataFrame([{
        'RETAILER_ID': retailer_id,
        'extracted_name': extracted_name
    }])
    
    # Apply the filtering for this single retailer
    filtered_bank_df = filter_bank_df_by_names(filtered_df, single_retailer_df)
    print(f"Found {len(filtered_bank_df)} matching rows for retailer {retailer_id}")
    
    if not filtered_bank_df.empty:
        # Assuming the credit column is named 'Credit' - adjust if different
        credit_sum = filtered_bank_df['Credit'].sum()
        print(f"Sum of Credit column for retailer {retailer_id}: {credit_sum}")
        
        # If you want to see the breakdown by retailer_id
        credit_by_retailer = filtered_bank_df.groupby('matched_retailer_id')['Credit'].sum().reset_index()
        if len(credit_by_retailer) == 1:  # Ensure there's only one row
            matched_retailer_id = credit_by_retailer.iloc[0]['matched_retailer_id']
            
            # Convert both to string for comparison to handle data type differences
            matched_retailer_id_str = str(matched_retailer_id)
            approve_filtered['RETAILER_ID_str'] = approve_filtered['RETAILER_ID'].astype(str)
            
            # Debug: Check data types and values
            print(f"matched_retailer_id type: {type(matched_retailer_id)}, value: {matched_retailer_id}")
            print(f"df_approve_filtered RETAILER_ID sample values: {approve_filtered['RETAILER_ID'].head().tolist()}")
            print(f"df_approve_filtered RETAILER_ID_str sample values: {approve_filtered['RETAILER_ID_str'].head().tolist()}")
            
            # Filter df_approve_filtered for matching RETAILER_ID
            matching_approve_rows = approve_filtered[approve_filtered['RETAILER_ID_str'] == matched_retailer_id_str]
            
            # Sum the REQUESTED_AMOUNT for matching rows
            requested_amount_sum = matching_approve_rows['REQUESTED_AMOUNT'].sum()
            
            print(f"Retailer ID: {matched_retailer_id}")
            print(f"Number of matching rows in df_approve_filtered: {len(matching_approve_rows)}")
            print(f"Sum of REQUESTED_AMOUNT: {requested_amount_sum}")
            
            # If you want to see the matching rows
            print("Matching rows from df_approve_filtered:")
            print(matching_approve_rows[['RETAILER_ID', 'REQUESTED_AMOUNT']])
            credit_by_retailer['Credit'] = credit_by_retailer['Credit'] - requested_amount_sum
            print(f"Updated credit_by_retailer after subtraction:")
            print(credit_by_retailer)
                # merge on retailer IDs
            data_for_gsheet = credit_by_retailer.merge(
            to_approve_list,
            left_on="matched_retailer_id",
            right_on="RETAILER_ID",
            how="left")

            if data_for_gsheet.iloc[0]['Credit'] > 0:
                google_sheets("Test_IPN", "Approved", "Append", df=data_for_gsheet)
            else:
                data_for_gsheet['Reason'] = 'No Credit match'
                google_sheets("Test_IPN", "Manual", "Append", df=data_for_gsheet)
            
            # Clean up the temporary column
            approve_filtered = approve_filtered.drop(columns=['RETAILER_ID_str'])
        else:
            print("Warning: credit_by_retailer has more than one row, expected only one")
    else:
        print(f"No matching rows found for retailer {retailer_id}")
    
    print("-" * 50)  # Separator between retailers








/home/ec2-user/service_account_key.json
/home/ec2-user/service_account_key.json
/home/ec2-user/service_account_key.json


/tmp/ipykernel_15166/3948141530.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Transaction Date'] = filtered_df['Transaction Date/Time'].dt.date
/tmp/ipykernel_15166/3948141530.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  approve_filtered['CREATED_DATE'] = approve_filtered['CREATED_AT'].dt.date
/tmp/ipykernel_15166/3948141530.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

/home/ec2-user/service_account_key.json
Total retailers in approve_list: 43
Retailers in check_data: 2
Retailers NOT in check_data: 41
Processing retailer ID: 1174964
Extracted names: HANY ABDALLAH ABDALLAH EMBABY
Found 0 matching rows for retailer 1174964
No matching rows found for retailer 1174964
--------------------------------------------------
Processing retailer ID: 1954912
Extracted names: AHMAD ABD ALSALAM FAOWZE
Found 2 matching rows for retailer 1954912
Sum of Credit column for retailer 1954912: 1000.0
matched_retailer_id type: <class 'str'>, value: 1954912
df_approve_filtered RETAILER_ID sample values: [1017688, 1838806, 1731629, 850806, 1814997]
df_approve_filtered RETAILER_ID_str sample values: ['1017688', '1838806', '1731629', '850806', '1814997']
Retailer ID: 1954912
Number of matching rows in df_approve_filtered: 1
Sum of REQUESTED_AMOUNT: 500.0
Matching rows from df_approve_filtered:
      RETAILER_ID  REQUESTED_AMOUNT
1099      1954912             500.0
Updated credi

In [4]:
check_data_with_names

,RETAILER_ID,extracted_name
0,1484889,حسام محمد على محمد
1,1736986,ايمن مسعود على محمد
2,1758955,ISLAM MOHAMED ELSAYED ABDEL HAMID
3,1977265,وليد شاكر على خميس
4,723064,Mohamed Abdelmoneam
5,912830,MAHMOUD SALAMA ABDEL MOAMEN HAMOUDA


In [2]:
def filter_bank_df_by_names(bank_df, check_data_with_names):
    """
    Filter bank_df to find rows where Description contains any of the names
    from the extracted_name column (split by ' | ')
    """
    filtered_rows = []
    
    for _, row in check_data_with_names.iterrows():
        retailer_id = row['RETAILER_ID']
        extracted_name = row['extracted_name']
        
        if pd.isna(extracted_name):
            continue
            
        # Handle both single names and multiple names separated by ' | '
        if ' | ' in str(extracted_name):
            # Split the extracted_name by ' | ' to get individual names
            names = [name.strip() for name in extracted_name.split(' | ')]
        else:
            # Single name case - no splitting needed
            names = [str(extracted_name).strip()]
        
        # Filter bank_df for this retailer's names
        for name in names:
            if name and name != 'nan':  # Skip empty names and NaN values
                matching_rows = bank_df[bank_df['Description'].str.contains(name, case=False, na=False)]
                if not matching_rows.empty:
                    # Add retailer_id to the matching rows
                    matching_rows = matching_rows.copy()
                    matching_rows['matched_retailer_id'] = retailer_id
                    matching_rows['matched_name'] = name
                    filtered_rows.append(matching_rows)
    
    if filtered_rows:
        return pd.concat(filtered_rows, ignore_index=True)
    else:
        return pd.DataFrame()